# Day 12 - o1

In [2]:
from collections import deque

def read_grid(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    return lines

def neighbors(r, c, rows, cols):
    for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        nr, nc = r + dr, c + dc
        yield nr, nc

def solve():
    grid = read_grid('input.txt')
    rows = len(grid)
    cols = len(grid[0])

    visited = [[False]*cols for _ in range(rows)]
    total_price = 0

    for r in range(rows):
        for c in range(cols):
            if not visited[r][c]:
                # BFS or DFS to find a region
                letter = grid[r][c]
                queue = deque([(r, c)])
                visited[r][c] = True
                area = 0
                perimeter = 0
                
                while queue:
                    rr, cc = queue.popleft()
                    area += 1
                    # Check neighbors for perimeter calculation
                    for nr, nc in neighbors(rr, cc, rows, cols):
                        if not (0 <= nr < rows and 0 <= nc < cols):
                            # Out of bounds -> contributes to perimeter
                            perimeter += 1
                        else:
                            if grid[nr][nc] == letter:
                                if not visited[nr][nc]:
                                    visited[nr][nc] = True
                                    queue.append((nr, nc))
                            else:
                                # Different letter -> contributes to perimeter
                                perimeter += 1

                # Price for this region
                price = area * perimeter
                total_price += price

    print(total_price)

if __name__ == "__main__":
    solve()


1449902


## Part 2

In [5]:
from collections import deque, defaultdict

def read_grid(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    return lines

def neighbors(r, c, rows, cols):
    for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        nr, nc = r + dr, c + dc
        yield nr, nc

def get_region_edges(region_cells, grid, rows, cols):
    # Determine all boundary edges for the region.
    # Edges are represented as ((r1,c1),(r2,c2)) with (r1,c1) < (r2,c2) lex order.
    edges = set()
    letter = grid[next(iter(region_cells))[0]][next(iter(region_cells))[1]]
    for (rr, cc) in region_cells:
        # Top edge
        if rr == 0 or grid[rr-1][cc] != letter or (rr-1, cc) not in region_cells:
            e = ((rr, cc), (rr, cc+1))
            if e[1] < e[0]:
                e = (e[1], e[0])
            edges.add(e)
        # Bottom edge
        if rr == rows-1 or grid[rr+1][cc] != letter or (rr+1, cc) not in region_cells:
            e = ((rr+1, cc), (rr+1, cc+1))
            if e[1] < e[0]:
                e = (e[1], e[0])
            edges.add(e)
        # Left edge
        if cc == 0 or grid[rr][cc-1] != letter or (rr, cc-1) not in region_cells:
            e = ((rr, cc), (rr+1, cc))
            if e[1] < e[0]:
                e = (e[1], e[0])
            edges.add(e)
        # Right edge
        if cc == cols-1 or grid[rr][cc+1] != letter or (rr, cc+1) not in region_cells:
            e = ((rr, cc+1), (rr+1, cc+1))
            if e[1] < e[0]:
                e = (e[1], e[0])
            edges.add(e)
    return edges

def count_sides(edges):
    # Build adjacency: vertex -> list of connected vertices (2 per vertex)
    adjacency = defaultdict(list)
    for e in edges:
        v1, v2 = e
        adjacency[v1].append(v2)
        adjacency[v2].append(v1)

    # Count corners:
    # Each vertex in a rectilinear polygon boundary has degree 2.
    # If the two edges connected at this vertex change direction (H->V or V->H), count it as a corner.
    sides = 0
    for v in adjacency:
        # v is a vertex: adjacency[v] has 2 vertices
        if len(adjacency[v]) != 2:
            # If input is consistent, this should never happen for a closed polygon
            continue
        v1, v2 = adjacency[v]
        r, c = v
        r1, c1 = v1
        r2, c2 = v2

        # Determine directions of edges (v->v1) and (v->v2)
        # Horizontal edge: same r, c differs
        # Vertical edge: same c, r differs
        dir1 = 'H' if r == r1 else 'V'
        dir2 = 'H' if r == r2 else 'V'
        if dir1 != dir2:
            sides += 1
    return sides

def solve():
    grid = read_grid('input.txt')
    rows = len(grid)
    cols = len(grid[0])

    visited = [[False]*cols for _ in range(rows)]
    total_price = 0

    for r in range(rows):
        for c in range(cols):
            if not visited[r][c]:
                # BFS to find a region
                letter = grid[r][c]
                queue = deque([(r, c)])
                visited[r][c] = True
                region_cells = {(r,c)}

                while queue:
                    rr, cc = queue.popleft()
                    for nr, nc in neighbors(rr, cc, rows, cols):
                        if 0 <= nr < rows and 0 <= nc < cols:
                            if not visited[nr][nc] and grid[nr][nc] == letter:
                                visited[nr][nc] = True
                                queue.append((nr, nc))
                                region_cells.add((nr, nc))

                area = len(region_cells)
                edges = get_region_edges(region_cells, grid, rows, cols)
                sides = count_sides(edges)
                price = area * sides
                total_price += price
    print(total_price)

if __name__ == "__main__":
    solve()


897612
